In [6]:
import os
from pathlib import Path

import dotenv
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)


ROOT_PATH = Path("python/translation/main.py").parent
OPENAI_ENV_PATH = ROOT_PATH.parent / "env" / "openai.env"
dotenv.load_dotenv("/Users/aiden/workspace/mlops-for-all/mlops-for-all.github.io/python/env/openai.env")

True

In [7]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = OpenAI(openai_api_key=OPENAI_API_KEY)

template="You are a helpful assistant that translates {input_language} to {output_language}. "
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt=PromptTemplate(
    template="You are a helpful assistant that translates {input_language} to {output_language}.",
    input_variables=["input_language", "output_language"],
)
system_message_prompt_2 = SystemMessagePromptTemplate(prompt=prompt)

assert system_message_prompt == system_message_prompt_2

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])


In [35]:
pred = llm.predict_messages([HumanMessage(content=f"Translate this sentence from Korean to English. {source_sentence}")])
pred

AIMessage(content='Summary of how Seldon Core creates an API server: \n\n1. initContainer downloads the necessary model from the model repository. \n2. The downloaded model is passed to the container. \n3. The container runs an API server that wraps the model. \n4. The API can be requested at the generated API server address to receive the inference values of the model. \n\nSeldonDeployment spec, which is the custom resource used most often when using Seldon Core: \n\nThe yaml file that defines the SeldonDeployment custom resource is as follows:', additional_kwargs={}, example=False)

In [38]:
print(pred.content)

Summary of how Seldon Core creates an API server: 

1. initContainer downloads the necessary model from the model repository. 
2. The downloaded model is passed to the container. 
3. The container runs an API server that wraps the model. 
4. The API can be requested at the generated API server address to receive the inference values of the model. 

SeldonDeployment spec, which is the custom resource used most often when using Seldon Core: 

The yaml file that defines the SeldonDeployment custom resource is as follows:


In [28]:

DOCS_ROOT = ROOT_PATH.parent.parent
source_path = "/Users/aiden/workspace/mlops-for-all/mlops-for-all.github.io/i18n/en/docusaurus-plugin-content-docs/version-1.0/api-deployment/seldon-fields.md"

translate_lines = []
with open(source_path, "r") as f:
    line = f.readline()
    translate_lines += [line]
    lines = []
    is_codeblock = False
    is_header = True
    while line:
        line = f.readline()
        # 헤더 블록인 경우
        if line.startswith("---"):
            is_header = False
            translate_lines += [line]
            continue
        if is_header:
            translate_lines += [line]
            continue

        # 코드 블록인 경우
        if line.startswith("```"):
            # 첫 번째 코드 블록인 경우 번역한다.
            if not is_codeblock:
                source_sentence = "".join(lines)
                if source_sentence:
                    #
                    # request
                    print(source_sentence)
                    translation_prompt = chat_prompt.format_prompt(input_language="English", output_language="Korean", text=source_sentence).to_messages()
                    print(translation_prompt)
                    # translated_sentence = chain.run(source_sentence)
                    translated_sentence = llm.predict_messages(translation_prompt)
                    print(translated_sentence)
                    #
                    translate_lines += [translated_sentence]
                    translate_lines += ["\n"]
                    break
                else:
                    translate_lines += ["\n"]
                lines = []
                is_codeblock = True
            else:
                is_codeblock = False
                translate_lines += [line]
                continue
        if is_codeblock:
            # 코드 블록 내부인 경우 통과한다.
            translate_lines += [line]
            continue
        lines += [line]

# source_sentence = "".join(lines)
# #
# # request
# translation_prompt = chat_prompt.format_prompt(input_language="English", output_language="Korean", text=source_sentence).to_messages()
# translated_sentence = llm.predict_messages(translation_prompt)
# #
# translate_lines += [translated_sentence]
# translate_lines += ["\n"]


## How Seldon Core works?

Seldon Core가 API 서버를 생성하는 과정을 요약하면 다음과 같습니다.

![seldon-fields-0.png](./img/seldon-fields-0.png)

1. initContainer는 모델 저장소에서 필요한 모델을 다운로드 받습니다.
2. 다운로드받은 모델을 container로 전달합니다.
3. container는 전달받은 모델을 감싼 API 서버를 실행합니다.
4. 생성된 API 서버 주소로 API를 요청하여 모델의 추론 값을 받을 수 있습니다.

## SeldonDeployment Spec

Seldon Core를 사용할 때, 주로 사용하게 되는 커스텀 리소스인 SeldonDeployment를 정의하는 yaml 파일은 다음과 같습니다.



[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}), HumanMessage(content='\n## How Seldon Core works?\n\nSeldon Core가 API 서버를 생성하는 과정을 요약하면 다음과 같습니다.\n\n![seldon-fields-0.png](./img/seldon-fields-0.png)\n\n1. initContainer는 모델 저장소에서 필요한 모델을 다운로드 받습니다.\n2. 다운로드받은 모델을 container로 전달합니다.\n3. container는 전달받은 모델을 감싼 API 서버를 실행합니다.\n4. 생성된 API 서버 주소로 API를 요청하여 모델의 추론 값을 받을 수 있습니다.\n\n## SeldonDeployment Spec\n\nSeldon Core를 사용할 때, 주로 사용하게 되는 커스텀 리소스인 SeldonDeployment를 정의하는 yaml 파일은 다음과 같습니다.\n\n\n', additional_kwargs={}, example=False)]

In [30]:
translation_prompt

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}),
 HumanMessage(content='\n## How Seldon Core works?\n\nSeldon Core가 API 서버를 생성하는 과정을 요약하면 다음과 같습니다.\n\n![seldon-fields-0.png](./img/seldon-fields-0.png)\n\n1. initContainer는 모델 저장소에서 필요한 모델을 다운로드 받습니다.\n2. 다운로드받은 모델을 container로 전달합니다.\n3. container는 전달받은 모델을 감싼 API 서버를 실행합니다.\n4. 생성된 API 서버 주소로 API를 요청하여 모델의 추론 값을 받을 수 있습니다.\n\n## SeldonDeployment Spec\n\nSeldon Core를 사용할 때, 주로 사용하게 되는 커스텀 리소스인 SeldonDeployment를 정의하는 yaml 파일은 다음과 같습니다.\n\n\n', additional_kwargs={}, example=False)]

In [31]:
llm.predict_messages(translation_prompt)

AIMessage(content='``` yaml\napiVersion: machinelearning.seldon.io/v1alpha2\nkind: SeldonDeployment\nmetadata:\n  name: seldon-model\nspec:\n  name: test-deployment\n  predictors:\n  - graph:\n      name: mymodel\n      type: MODEL\n      modelUri: gs://my-bucket/model.pkl\n      endpoint:\n      - type: REST\n        servicePort: 80\n```\n\n이 yaml 파일은 SeldonDeployment를 정의하고 모델이 저장된 저장소의 위치를 지정하고, Seldon Core가 생성할 API 서버의 포트번호를 지정합니다.', additional_kwargs={}, example=False)

In [26]:
print(translated_sentence.content)

```
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: <커스텀 리소스 이름>
spec:
  name: <모델 이름>
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: <컨테이너 이미지>
    graph:
      children: []
      endpoint:
        type: REST
      name: <모델 이름>
      type: MODEL
```

SeldonDeployment는 모델을 배포하기 위해, 컨테이너 이미지, 모델 이름, 등의 리소스


In [ ]:
translated_sentence

In [ ]:


docs = "".join(translate_lines)
print(docs)
dest_path = DOCS_ROOT/ "i18n/en/docusaurus-plugin-content-docs/version-1.0/api-deployment" / "seldon-fields.md"
with open(dest_path, "w") as f:
    f.write(docs)
